In [ ]:
!pip install sklearn

In [1]:
import keras
import pandas as pd
import sklearn 
from sklearn.model_selection import train_test_split
import numpy as np

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv('data/train.csv')

In [3]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X_train = df.drop("label", axis=1)
labels = df["label"].copy().values
possible_labels = [a for a in range(10)]

In [5]:
print(df["label"].unique())
print([a for a in range(10)])

[1 0 4 7 3 5 8 9 2 6]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [6]:
X_train = X_train.values.reshape(len(X_train),28,28)

In [7]:
X_train.shape

(42000, 28, 28)

In [8]:
XX_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X_train,labels, random_state=42 , stratify=labels)
print(XX_train.shape,X_val.shape, y_train.shape, y_val.shape)

(31500, 28, 28) (10500, 28, 28) (31500,) (10500,)


In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K


### constants

In [33]:
batch_size = 128
num_classes = 10
epochs = 10
# input image dimensions
img_rows, img_cols = 28, 28

In [11]:
 K.image_data_format() 

'channels_last'

### reshape training as matrix of 28x28x1 channel (rgb but here it is black and white) 

In [12]:
XX_train = XX_train.reshape(XX_train.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [13]:
XX_train.shape

(31500, 28, 28, 1)

### feature scaling

In [14]:
XX_train = XX_train.astype('float32')
X_val = X_val.astype('float32')
XX_train /= 255
X_val /= 255
print('XX_train shape:', XX_train.shape, XX_train[0].max(), XX_train[0].min())
print(XX_train.shape[0], 'train samples')
print(X_val.shape[0], 'test samples')

XX_train shape: (31500, 28, 28, 1) 1.0 0.0
31500 train samples
10500 test samples


In [15]:
# convert class vectors to binary class matrices
y_train_cat = keras.utils.to_categorical(y_train, num_classes)
y_val_cat = keras.utils.to_categorical(y_val, num_classes)

In [16]:
print(y_train_cat[1], y_train[1])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.] 2


### model definition

In [34]:
model = Sequential()
#Conv2D channel
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


### training

In [35]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model.fit(XX_train, y_train_cat,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val_cat))

Train on 31500 samples, validate on 10500 samples
Epoch 1/10
31500/31500 [==============================] - 50s 2ms/step - loss: 0.4398 - acc: 0.8702 - val_loss: 0.1407 - val_acc: 0.9565
Epoch 2/10
31500/31500 [==============================] - 49s 2ms/step - loss: 0.1649 - acc: 0.9517 - val_loss: 0.0951 - val_acc: 0.9717
Epoch 3/10
31500/31500 [==============================] - 50s 2ms/step - loss: 0.1136 - acc: 0.9659 - val_loss: 0.0794 - val_acc: 0.9746
Epoch 4/10
31500/31500 [==============================] - 49s 2ms/step - loss: 0.0943 - acc: 0.9723 - val_loss: 0.0660 - val_acc: 0.9787
Epoch 5/10
31500/31500 [==============================] - 49s 2ms/step - loss: 0.0823 - acc: 0.9743 - val_loss: 0.0582 - val_acc: 0.9824
Epoch 6/10
31500/31500 [==============================] - 49s 2ms/step - loss: 0.0703 - acc: 0.9786 - val_loss: 0.0584 - val_acc: 0.9818
Epoch 7/10
 4608/31500 [===>..........................] - ETA: 43s - loss: 0.0558 - acc: 0.9809

### make predictions

In [20]:
dT = pd.read_csv('data/test.csv')

In [21]:
dT.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
X_test = dT.values.reshape(len(dT),28,28)
XX_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

In [23]:
XX_test = XX_test.astype('float32')
XX_test /= 255
XX_test.shape

(28000, 28, 28, 1)

In [ ]:
dExemple = pd.read_csv('sample_submission.csv')

In [25]:
predictions = model.predict(XX_test)

In [26]:
predictions[0]

array([4.3903319e-06, 3.0740512e-07, 9.9987471e-01, 7.9664591e-05,
       1.7068031e-07, 3.3250748e-07, 2.0608924e-08, 6.5521212e-06,
       3.3472763e-05, 4.5094919e-07], dtype=float32)

In [28]:
np.argmax(predictions[4])

3

In [31]:
predictions_dataframe = [ (i+1, np.argmax(p)) for i, p in enumerate(predictions)]
results_df = pd.DataFrame(predictions_dataframe, columns=['ImageId','Label'])
results_df["Label"].value_counts()

1    3210
7    2894
8    2810
2    2773
9    2772
0    2766
4    2760
3    2734
6    2712
5    2569
Name: Label, dtype: int64

In [ ]:
results_df.to_csv('submit1.csv',index=False)
r = pd.read_csv('submit.csv')
r.head()